In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head(n=891)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
print(train_data["Name"])

0                                Braund, Mr. Owen Harris
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                 Heikkinen, Miss. Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                               Allen, Mr. William Henry
                             ...                        
886                                Montvila, Rev. Juozas
887                         Graham, Miss. Margaret Edith
888             Johnston, Miss. Catherine Helen "Carrie"
889                                Behr, Mr. Karl Howell
890                                  Dooley, Mr. Patrick
Name: Name, Length: 891, dtype: object


In [4]:
train_data["cabin_letter"]=train_data.Cabin.apply(lambda x: str(x)[0])
#print(train_data["cabin_letter"])
train_data["name_title"]=train_data.Name.apply(lambda x: x.split(",")[1].split(".")[0])
#print(train_data["name_title"])
train_data["norm_fare"]=np.log(train_data["Fare"]+1)
print(train_data["norm_fare"])

0      2.110213
1      4.280593
2      2.188856
3      3.990834
4      2.202765
         ...   
886    2.639057
887    3.433987
888    3.196630
889    3.433987
890    2.169054
Name: norm_fare, Length: 891, dtype: float64


In [5]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')
test_data["cabin_letter"]=test_data.Cabin.apply(lambda x: str(x)[0])
#test_data["name_title"]=test_data.Name.apply(lambda x: x.split(",")[1].split(".")[0])
test_data["norm_fare"]=np.log(train_data["Fare"]+1)

In [6]:
#XGBoost!
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

y = train_data["Survived"]

#train_data.dropna(subset=["Age"],inplace=True)

features = ["Pclass","Sex","cabin_letter","norm_fare"]#"SibSp", "Parch","name_title"]#age and embarked have NAs and crash things. dropping dosn't help
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = XGBClassifier(random_state=1,max_depth=5,reg_lambda=2,reg_alpha=1)#n_estimators=100, max_depth=20, 
model.fit(X, y)

cv=cross_val_score(model,X,y,cv=10)
print(cv.mean())

0.8193508114856429


In [7]:
from sklearn.model_selection import GridSearchCV #more comprehensive but slower
from sklearn.model_selection import RandomizedSearchCV
xgb = XGBClassifier(random_state = 1,max_depth=5,reg_lambda=2,reg_alpha=1)

param_grid = {
    'n_estimators': [100,200],
    'colsample_bytree': [0.75,0.8],
    'max_depth': [3,4,5],
    'reg_alpha': [1],
    'reg_lambda': [5, 10],
    'subsample': [0.55, 0.6],
    'learning_rate':[0.3],
    'gamma':[.5,1],
    'min_child_weight':[0.01],
    'sampling_method': ['uniform']
}

clf_xgb = RandomizedSearchCV(xgb, param_distributions = param_grid, cv = 7, verbose = True, n_jobs = -1)
best_clf_xgb = clf_xgb.fit(X,y)


Fitting 7 folds for each of 10 candidates, totalling 70 fits


In [8]:
X.head()

,Pclass,norm_fare,Sex_female,Sex_male,cabin_letter_A,cabin_letter_B,cabin_letter_C,cabin_letter_D,cabin_letter_E,cabin_letter_F,cabin_letter_G,cabin_letter_T,cabin_letter_n
0,3,2.110213,0,1,0,0,0,0,0,0,0,0,1
1,1,4.280593,1,0,0,0,1,0,0,0,0,0,0
2,3,2.188856,1,0,0,0,0,0,0,0,0,0,1
3,1,3.990834,1,0,0,0,1,0,0,0,0,0,0
4,3,2.202765,0,1,0,0,0,0,0,0,0,0,1


In [9]:
X_test.head()

,Pclass,norm_fare,Sex_female,Sex_male,cabin_letter_A,cabin_letter_B,cabin_letter_C,cabin_letter_D,cabin_letter_E,cabin_letter_F,cabin_letter_G,cabin_letter_n
0,3,2.110213,0,1,0,0,0,0,0,0,0,1
1,3,4.280593,1,0,0,0,0,0,0,0,0,1
2,2,2.188856,0,1,0,0,0,0,0,0,0,1
3,3,3.990834,0,1,0,0,0,0,0,0,0,1
4,3,2.202765,1,0,0,0,0,0,0,0,0,1


In [10]:
X_test["cabin_letter_T"]=0

y_hat_xgb = best_clf_xgb.best_estimator_.predict(X_test).astype(int)
xgb_submission = {'PassengerId': test_data.PassengerId, 'Survived': y_hat_xgb}
submission_xgb = pd.DataFrame(data=xgb_submission)
submission_xgb.to_csv('csn_xgb_submission_tryMaxDepthSearch_dialdown.csv', index=False)
submission_xgb.to_csv('submission.csv', index=False)